## 导入必要的库

In [ ]:
import torch

## 1. Eager Mode（近似 Unboxed 行为）

虽然是从 Python 调用的，但 PyTorch 的 Python 绑定经过高度优化，会尽可能直接调用底层的 C++ 函数 (at::native::mul)，开销较小。

In [ ]:
def eager_mul(x, y):
    return torch.mul(x, y)

## 2. TorchScript Mode（Boxed 行为的典型）

JIT 编译后，函数变成了一个 Graph。执行时，JIT 解释器不仅不知道具体的 C++ 签名，也不想为每个算子写特定的调用逻辑。它会将 x, y 打包成 IValue 放入 Stack，通过 OperatorHandle 查找 "aten::mul"，然后调用其 Boxed Kernel（通常是一个通用的 wrapper）。

In [ ]:
scripted_mul = torch.jit.script(eager_mul)

## 测试和比较

In [ ]:
x = torch.randn(100, 100)
y = torch.randn(100, 100)

print("--- Eager Mode (Unboxed Path dominant) ---")
# 这里 Python 直接绑定到 C++ 函数入口
print(eager_mul(x, y).shape)

In [ ]:
print("\n--- TorchScript Mode (Boxed Path mechanism) ---")
# 打印 JIT 图，可以看到 aten::mul 被作为一个节点存储
print(scripted_mul.graph)

In [ ]:
# 实际执行时：
# 1. 解释器将 x, y 压入 Stack (std::vector<IValue>)
# 2. 获取 aten::mul 的 OperatorHandle
# 3. Handle.callBoxed(stack) -> 解析 Stack 参数 -> 调用实际内核
print(scripted_mul(x, y).shape)

## 进阶：显式通过 torch.ops 调用

`torch.ops` 是 PyTorch 暴露给 Python 的 OperatorHandle 集合。当你调用 `torch.ops.aten.mul` 时，你实际上是在与 Dispatcher 交互，寻找名为 aten::mul 的算子句柄。这种调用方式在内部机制上更接近于一种动态查找过程。

In [ ]:
print("\n--- 显式 OperatorHandle 调用 ---")
op_handle = torch.ops.aten.mul
print(f"算子句柄: {op_handle}")
print(op_handle(x, y).shape)